<a href="https://colab.research.google.com/github/Advanced-Research-Centre/QraftRNA/blob/main/Grover's%20search%20for%20RNA%20design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grover's serach for RNA design

In [ ]:
import numpy as np
from math import pi,log2,floor,ceil
from qiskit import *
from qiskit.circuit import *
from qiskit.extensions import *
from qiskit.circuit.library import *
from qiskit.extensions.simulator.snapshot import snapshot
from qiskit.quantum_info.operators import Operator
from qiskit.extensions.simulator.snapshot import snapshot
from scipy import optimize
from matplotlib.pyplot import plot,show
%matplotlib inline
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

## 1 - Addition

Let's implement the circuit in [1] modified to add mod 2^n.

In [ ]:
#MAJ
q=QuantumRegister(3)
MAJ = QuantumCircuit(q)

MAJ.cnot(q[2],q[1])
MAJ.cnot(q[2],q[0])
MAJ.ccx(q[0],q[1],q[2])

maj = MAJ.to_gate(label='MAJ')

#UMA
q=QuantumRegister(3)
UMA = QuantumCircuit(q)

UMA.ccx(q[0],q[1],q[2])
UMA.cnot(q[2],q[0])
UMA.cnot(q[0],q[1])

uma = UMA.to_gate(label='UMA')

#add
def add_circuit(nb_bits):
    concerved = QuantumRegister(nb_bits, name="=")
    not_concerved = QuantumRegister(nb_bits, name="->")
    initial_carry = QuantumRegister(1)
    add = QuantumCircuit(initial_carry,concerved,not_concerved)
    add.append(maj,[initial_carry,not_concerved[0],concerved[0]])
    for i in range(1,nb_bits):
        add.append(maj,[concerved[i-1],not_concerved[i],concerved[i]])
    for i in range(nb_bits-1,0,-1):
        add.append(uma,[concerved[i-1],not_concerved[i],concerved[i]])
    add.append(uma,[initial_carry,not_concerved[0],concerved[0]])
    return add

def add_gate(nb_bits):
    return add_circuit(nb_bits).to_gate(label="add{}".format(nb_bits))

add_circuit(6).draw()

┌──────┐                                                                »
q1499: ┤0     ├────────────────────────────────────────────────────────────────»
       │      │┌──────┐                                                        »
  =_0: ┤2     ├┤0     ├────────────────────────────────────────────────────────»
       │      ││      │┌──────┐                                                »
  =_1: ┤      ├┤2     ├┤0     ├────────────────────────────────────────────────»
       │      ││      ││      │┌──────┐                                ┌──────┐»
  =_2: ┤      ├┤      ├┤2     ├┤0     ├────────────────────────────────┤0     ├»
       │  MAJ ││      ││      ││      │┌──────┐                ┌──────┐│      │»
  =_3: ┤      ├┤      ├┤      ├┤2     ├┤0     ├────────────────┤0     ├┤2     ├»
       │      ││  MAJ ││      ││      ││      │┌──────┐┌──────┐│      ││      │»
  =_4: ┤      ├┤      ├┤      ├┤      ├┤2     ├┤0     ├┤0     ├┤2     ├┤      ├»
       │      ││      ││  MAJ ││      ││      ││      ││      ││      ││      │»
  =_5: ┤      ├┤      ├┤      ├┤      ├┤      ├┤2     ├┤2     ├┤      ├┤      ├»
       │      ││      ││      ││  MAJ ││      ││      ││      ││      ││  UMA │»
 ->_0: ┤1     ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├»
       └──────┘│      ││      ││      ││  MAJ ││      ││      ││  UMA ││      │»
 ->_1: ────────┤1     ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├»
               └──────┘│      ││      ││      ││  MAJ ││  UMA ││      ││      │»
 ->_2: ────────────────┤1     ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├»
                       └──────┘│      ││      ││      ││      ││      ││      │»
 ->_3: ────────────────────────┤1     ├┤      ├┤      ├┤      ├┤      ├┤1     ├»
                               └──────┘│      ││      ││      ││      │└──────┘»
 ->_4: ────────────────────────────────┤1     ├┤      ├┤      ├┤1     ├────────»
                                       └──────┘│      ││      │└──────┘        »
 ->_5: ────────────────────────────────────────┤1     ├┤1     ├────────────────»
                                               └──────┘└──────┘                »
«                       ┌──────┐
«q1499: ────────────────┤0     ├
«               ┌──────┐│      │
«  =_0: ────────┤0     ├┤2     ├
«       ┌──────┐│      ││      │
«  =_1: ┤0     ├┤2     ├┤      ├
«       │      ││      ││      │
«  =_2: ┤2     ├┤      ├┤      ├
«       │      ││      ││  UMA │
«  =_3: ┤      ├┤      ├┤      ├
«       │      ││  UMA ││      │
«  =_4: ┤      ├┤      ├┤      ├
«       │  UMA ││      ││      │
«  =_5: ┤      ├┤      ├┤      ├
«       │      ││      ││      │
« ->_0: ┤      ├┤      ├┤1     ├
«       │      ││      │└──────┘
« ->_1: ┤      ├┤1     ├────────
«       │      │└──────┘        
« ->_2: ┤1     ├────────────────
«       └──────┘                
« ->_3: ────────────────────────
«                               
« ->_4: ────────────────────────
«                               
« ->_5: ────────────────────────
«

## 2 - Matchnig parentesis

`(`, `.` and `)` are encoded respectively by 1, 0 and -1, which are encoded with one's complement representation.

In [ ]:
nb_bits_bracket_dot = 2
nb_bits_loop_type = 3

def n_to_ctrl_state(n,nb_bits):
    return format(n,"0{}b".format(nb_bits))

#if test_concerved == tested_to_0 then tested_to_0 is set to 0 and will trigger the application of some gate in then_append(circuit,tested_to_0,nb_bits_test,gate,...)
def start_if_equal(circuit,tested_to_0,nb_bits_test,test_concerved):
    for i in range(nb_bits_test):
        circuit.cx(test_concerved[i],tested_to_0[i])

end_if_equal=start_if_equal

def then_append(circuit,test_0,nb_bits_test,gate,qbits_list):
    circuit.append(gate.control(nb_bits_test,ctrl_state="0"*nb_bits_test),[test_0[i] for i in range(nb_bits_test)]+qbits_list)

def matching_parenthesis_circuit(length):
    nb_bits_position=ceil(log2(length+1)) #length+1 because we add a ( at the begining to check the folding is well formed
    add_positions=add_gate(nb_bits_position)

    #input
    #semantics of folding
    #. == "00"
    #h == "10"
    #) == "11"
    #( == "01"
    folding = QuantumRegister(nb_bits_position,name="(/./h/)")

    #auxillary qbits
    k_folding = QuantumRegister(nb_bits_position,name="k_folding")
    count = QuantumRegister(nb_bits_position,name="count") #count how many parenthesis still need to be closed. count[k+1] corresponds to folding[k] and count[0] is initiallised to ( and is used to check that there is no extra ) that would match in the folding
    k_count = QuantumRegister(nb_bits_position,name="k_count")
    continue_searching_right_matching_left = QuantumRegister(1,name="continue_match") #continue_searching_right_matching_left[k] corresponds to count[k]

    change_folding_encoding = QuantumRegister(1,name="change_encoding")
    continue_scanning = QuantumRegister(1,name="continue_scan")
    zeros = QuantumRegister(nb_bits_position,name="zero")
    one = QuantumRegister(1,name="one")

    #results
    k_right_matching_left = QuantumRegister(nb_bits_position,name="k_match") #k_right_matching_left[k_left] is initialised to k_left and is incremented until count[k_left+1]==0, that is until k_right_matching_left[k_left] is the indice of the ) matching the ( ~ folding[k_left-1]
    loop = QuantumRegister(nb_bits_position,name="loop") #length of the loop starting at folding[k_left]==(. loop[k_left] is initialised to 0 and is incremented each time count[k_left]==1 until count[k_left]==0
    #semantics of loop_type
    #interior -> ?
    #buggle   -> ?
    #multi    -> ?
    #hairpin  -> ?
    #stack    -> ?
    loop_type = QuantumRegister(nb_bits_loop_type,name="loop_type")
    incorrect_folding = QuantumRegister(nb_bits_position,name="incorrect")
    correct_folding = QuantumRegister(1,name="correct")

    circuit = QuantumCircuit(change_folding_encoding,folding,k_folding,count,k_count,continue_scanning,continue_searching_right_matching_left,one,zeros,k_right_matching_left,loop,loop_type,incorrect_folding,correct_folding)


    #1) Superposition at the begining of the Groover search

    #initialize k_kolding
    circuit.h(k_folding)

    #any symbol at any position
    for i in range(nb_bits_bracket_dot):
        circuit.h(folding[i])

    #except at position "0"*nb_bits_position, where it is necessarily a ( == "11" to check if the folding is well formed
    for i in range(nb_bits_bracket_dot):
        circuit.append(HGate().control(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[k_folding[j] for j in range(nb_bits_position)]+[folding[i]])
        circuit.append(MCXGate(        nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[k_folding[j] for j in range(nb_bits_position)]+[folding[i]])

    #except at positions k > length, where folding[k] is set to .
    for k in range(length+1,2**nb_bits_position):
        for i in range(nb_bits_bracket_dot):
            circuit.append(HGate().control(nb_bits_position,ctrl_state=n_to_ctrl_state(k,nb_bits_position)),[k_folding[j] for j in range(nb_bits_position)]+[folding[i]])

    #if this symbol is h == "10", then change it to be . == "00"
    #OPTIMISATION here ?
    circuit.ccx(*[folding[i] for i in range(nb_bits_bracket_dot)],change_folding_encoding,ctrl_state="10")
    circuit.cnot(change_folding_encoding,folding[nb_bits_bracket_dot-1])


    #2) Initialisation of the oracle

    #initialise one to 1
    circuit.x(one)

    #when folding[k_left] == ( (which will be checked by there <*>) then there is initially 1 parenthesis to be closed
    circuit.x(continue_searching_right_matching_left)

    #initialise k_count to k_left
    circuit.h(k_count)
    #initialise k_right_matching_left[k_count] to k_left (and it will then be incremented up to the matchning k_right)
    for i in range(nb_bits_position):
        circuit.cx(k_count[i],k_right_matching_left[i])

    #extend folding
    #folding[0:nb_bits_bracket_dot] -> folding[0:nb_bits_position]
    #. == "00" ->  0 == "0...000"
    #do nothing
    #( == "01" ->  1 == "0...001"
    #do nothing
    #) == "11" -> -1 == "1...111"
    if nb_bits_bracket_dot < nb_bits_position:
        circuit.append(MCMT(XGate(),nb_bits_bracket_dot,nb_bits_position-nb_bits_bracket_dot),[folding[i] for i in range(nb_bits_position)])

    #if k_folding == k_count
    start_if_equal(circuit,k_count,nb_bits_position,k_folding)

    #initialise count with folding
    #folding -> count
    #. == "00" -> 0 == "0...000"
    #do nothing
    #) == "11" -> 0 == "0...000"
    #do nothing
    #( == "01" -> 1 == "0...001"
    then_append(circuit,k_count,nb_bits_position,CXGate(),[folding[0],count[0]])

    #end if k_folding == k_count
    end_if_equal(circuit,k_count,nb_bits_position,k_folding)

    #initialise continue_searching_right_matching_left=1 when count == (
    circuit.cnot(count[0],continue_searching_right_matching_left)

    #initialise continue_scanning=1
    circuit.x(continue_scanning)


    #3) Compute parameters by scanning the folding

    for k in range(length):
        #<*> if it has been checked that there is a matching ) to find, then continue_searching_right_matching_left[k_left]==1, otherwise continue_searching_right_matching_left[k_left]==0

        #increment k_right_matching_left[k_left] if continue_searching_right_matching_left[k_left]==1
        circuit.append(add_positions.control(),[continue_scanning,zeros[nb_bits_position-1],continue_searching_right_matching_left]+[zeros[i] for i in range(nb_bits_position-1)]+[k_right_matching_left[i] for i in range(nb_bits_position)])

        #go to the next position
        #increment k_count
        circuit.append(add_positions,[zeros[nb_bits_position-1],one]+[zeros[i] for i in range(nb_bits_position-1)]+[k_count[i] for i in range(nb_bits_position)])

        #if the end has been reached, that is if k_count == 0
        #then do not continue_scanning
        circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[k_count[i] for i in range(nb_bits_position)]+[continue_scanning])

        #if k_folding == k_count
        start_if_equal(circuit,k_count,nb_bits_position,k_folding)

        #if continue_searching_right_matching_left[k_left]==1, then update count[k_left] for the next candidate
        if 0 == k:
            #necessarily if folding[k_left]==( then continue_searching_right_matching_left[k_left]==1
            #so it is ok not controlling by continue_searching_right_matching_left here
            then_append(circuit,k_count,nb_bits_position,add_positions.control()              ,[continue_scanning,zeros[nb_bits_position-1]]+[count[i] for i in range(nb_bits_position)]+[folding[i] for i in range(nb_bits_position)])
        else:
            then_append(circuit,k_count,nb_bits_position,add_positions.control(2)             ,[continue_scanning,continue_searching_right_matching_left,zeros[nb_bits_position-1]]+[count[i] for i in range(nb_bits_position)]+[folding[i] for i in range(nb_bits_position)])


        #if continue_searching_right_matching_left[k_left]==0, then simply increment count[k_left] so that it never equals 0 again
        #it is ok not controlling by continue_scanning here
        if 0 != k:
            #as stated previously, if 0==k then necessarily if folding[k_left]==( then continue_searching_right_matching_left[k_left]==1
            #so there is no need doing this if 0==k
            then_append(circuit,k_count,nb_bits_position,add_positions.control(ctrl_state="0"),[continue_searching_right_matching_left,zeros[nb_bits_position-1]]+[count[i] for i in range(nb_bits_position)]+[one]+[zeros[i] for i in range(nb_bits_position-1)])

        #if 1 == count[k_left]
        start_if_equal(circuit,count,nb_bits_position,[one]+[zeros[i] for i in range(nb_bits_position-1)])

        #if 1 == continue_searching_right_matching_left[k_left] then increment loop[k_left]
        if 0==k:
            #since the value 0==loop[k_left] is known then there is no need to add in order to increment
            #a CXGate() is sufficient
            then_append(circuit,[k_count[i] for i in range(nb_bits_position)]+[count[i] for i in range(nb_bits_position)],2*nb_bits_position,CCXGate(),[continue_scanning,continue_searching_right_matching_left,loop[0]])
        else:
            then_append(circuit,[k_count[i] for i in range(nb_bits_position)]+[count[i] for i in range(nb_bits_position)],2*nb_bits_position,add_positions.control(),[continue_scanning,zeros[nb_bits_position-1]]+[loop[i] for i in range(nb_bits_position)]+[continue_searching_right_matching_left]+[zeros[i] for i in range(nb_bits_position-1)])

        #end if 1 == count[k_left]
        end_if_equal(circuit,count,nb_bits_position,[one]+[zeros[i] for i in range(nb_bits_position-1)])
        #end if k_folding == k_count
        end_if_equal(circuit,k_count,nb_bits_position,k_folding)

        #if count[k_left] == 0
        #then there is no matching ) to find
        circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[count[i] for i in range(nb_bits_position)]+[continue_searching_right_matching_left])

    #set k_count to its initial value by adding it 2**nb_bits_position - length
    first_one = -1
    for i,bit in enumerate(n_to_ctrl_state(2**nb_bits_position - length,nb_bits_position)):
        if '1'==bit:
            if 0 > first_one:
                first_one = i
            else:
                circuit.x(zeros[i])
    circuit.append(add_positions,[zeros[first_one]]+[k_count[i] for i in range(nb_bits_position)]+[(one if first_one == i else zeros[i]) for i in range(nb_bits_position)])

    #restore zeros
    first_one = -1
    for i,bit in enumerate(n_to_ctrl_state(2**nb_bits_position - length,nb_bits_position)):
        if '1'==bit:
            if 0 > first_one:
                first_one = i
            else:
                circuit.x(zeros[i])

    #Computed parameters
    #folding[kleft] -> continue_searching_right_matching_left[k_left] * k_right_matching_left[k_left] * loop                                 * k_count[k_left] * count[k_left]
    #(              -> 0 if a matching ) has been found               * k_right_matching_left         * number of . (at level 1) in the loop * k_left          * unspecified
    #(              -> 1 if a matching ) has not been found           * !=k_left                      * unspecified                          * k_left          * unspecified
    #. or )         -> 0                                              * k_left                        * 0                                    * k_left          * unspecified


    #4) Aggregating results

    #if each ( of the folding has a matching ) and if an extra ( at the beginning does not have a matching ), then it is a correst folding

    #checking if there is an exceeding )
    #there is if 0 == continue_searching_right_matching_left[k_left=0]
    #since the value 0==incorrect_folding is known then there is no need to add in order to increment
    #a CXGate() is sufficient
    circuit.append(MCXGate(nb_bits_position+1,ctrl_state="0"+n_to_ctrl_state(0,nb_bits_position)),[k_count[i] for i in range(nb_bits_position)]+[continue_searching_right_matching_left,incorrect_folding[0]])

    #sum all the incorrect flags continue_searching_right_matching_left (1 per position) into incorrect_folding
    #the sum cannot overflow since if the flag continue_searching_right_matching_left[k_left=0] == 0 (at k_left=0 incorrect beeing 0) then there is a ) somewhere
    #so continue_searching_right_matching_left[k_left=somewhere] == 0
    #so incorrect_folding is not incremented at each position
    for k in range(1,length+1):
        circuit.append(add_positions.control(nb_bits_position,ctrl_state=n_to_ctrl_state(k,nb_bits_position)),[k_count[i] for i in range(nb_bits_position)]+[zeros[nb_bits_position-1],continue_searching_right_matching_left]+[zeros[i] for i in range(nb_bits_position-1)]+[incorrect_folding[i] for i in range(nb_bits_position)])

    circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[incorrect_folding[i] for i in range(nb_bits_position)]+[correct_folding])

    return circuit

def matching_parenthesis_gate(length):
    return matching_parenthesis_circuit(length).to_gate(label="matching")

circ=matching_parenthesis_circuit(3)

circ.draw()


┌───┐                         »
change_encoding: ─────────────────────────┤ X ├──■──────────────────────»
                 ┌───┐┌───┐┌───┐          └─┬─┘  │                      »
      (/./h/)_0: ┤ H ├┤ H ├┤ X ├────────────o────┼────■─────────────────»
                 ├───┤└─┬─┘└─┬─┘┌───┐┌───┐  │  ┌─┴─┐  │                 »
      (/./h/)_1: ┤ H ├──┼────┼──┤ H ├┤ X ├──■──┤ X ├──┼─────────────────»
                 ├───┤  │    │  └─┬─┘└─┬─┘     └───┘  │                 »
    k_folding_0: ┤ H ├──o────o────o────o────■─────────┼────■────────────»
                 ├───┤  │    │    │    │    │         │    │            »
    k_folding_1: ┤ H ├──o────o────o────o────┼────■────┼────┼────■───────»
                 └───┘                      │    │  ┌─┴─┐  │    │       »
        count_0: ───────────────────────────┼────┼──┤ X ├──┼────┼────■──»
                                            │    │  └─┬─┘  │    │    │  »
        count_1: ───────────────────────────┼────┼────┼────┼────┼────┼──»
                 ┌───┐                    ┌─┴─┐  │    │  ┌─┴─┐  │    │  »
      k_count_0: ┤ H ├──■─────────────────┤ X ├──┼────o──┤ X ├──┼────┼──»
                 ├───┤  │                 └───┘┌─┴─┐  │  └───┘┌─┴─┐  │  »
      k_count_1: ┤ H ├──┼────■─────────────────┤ X ├──o───────┤ X ├──┼──»
                 ├───┤  │    │                 └───┘          └───┘  │  »
  continue_scan: ┤ X ├──┼────┼───────────────────────────────────────┼──»
                 ├───┤  │    │                                     ┌─┴─┐»
 continue_match: ┤ X ├──┼────┼─────────────────────────────────────┤ X ├»
                 ├───┤  │    │                                     └───┘»
            one: ┤ X ├──┼────┼──────────────────────────────────────────»
                 └───┘  │    │                                          »
         zero_0: ───────┼────┼──────────────────────────────────────────»
                        │    │                                          »
         zero_1: ───────┼────┼──────────────────────────────────────────»
                      ┌─┴─┐  │                                          »
      k_match_0: ─────┤ X ├──┼──────────────────────────────────────────»
                      └───┘┌─┴─┐                                        »
      k_match_1: ──────────┤ X ├────────────────────────────────────────»
                           └───┘                                        »
         loop_0: ───────────────────────────────────────────────────────»
                                                                        »
         loop_1: ───────────────────────────────────────────────────────»
                                                                        »
    loop_type_0: ───────────────────────────────────────────────────────»
                                                                        »
    loop_type_1: ───────────────────────────────────────────────────────»
                                                                        »
    loop_type_2: ───────────────────────────────────────────────────────»
                                                                        »
    incorrect_0: ───────────────────────────────────────────────────────»
                                                                        »
    incorrect_1: ───────────────────────────────────────────────────────»
                                                                        »
        correct: ───────────────────────────────────────────────────────»
                                                                        »
«                                                                          »
«change_encoding: ─────────────────────────────────────────────────────────»
«                                                  ┌─────────────────┐     »
«      (/./h/)_0: ─────────────────────────────────┤3                ├─────»
«                                                  │                 │     »
«      (/./h/)_1: ─────────────

statistics

In [ ]:
print("width {} ; depth {}".format(circ.width(),circ.depth()))

width 24 ; depth 42
